In [ ]:
# This script allows to generate a N-sample daily power consumption profile,
# for each of the data test days and for each of the nb_init decoders created with cvae_generate.ipynb

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import os
import time
import datetime
import numpy as np
import glob
from numpy import zeros
from numpy import ones

tf.keras.backend.set_floatx('float64')
tf.config.experimental_run_functions_eagerly(True)
from cvae import CVAE, simulate_cvae
import random

In [ ]:
# select the daily exogeous variables used to train CVAEs:  
#       rescaled temperatures (3 temperatures computed using a dimensionallity reduction method - PCA)
#       a smooth rescaled temperature 
#       position in the year (0 on January 1., 1 on December, 31.)
#       working day or not 
#       tariff daily profile (48 * 2 = 96 variables : 48 with 1 if tariff is Low, 0 if not and 48 with 1 if tariff is High, 0 if not)

cols_data = []
cols_price = []
cols_temp = ['temp_smooth_n']

for i in range(48):
    cols_data.append('consumption_{:02}'.format(i))
    cols_price.append('low_{:02}'.format(i))
    cols_price.append('high_{:02}'.format(i))

for i in range(3):
    cols_temp.append('pca_temp_{:1}'.format(i))
cols_cond = cols_temp + ['pos_y', 'work_day'] + cols_price 
cond_dim = len(cols_cond) 

input_dim = 48
latent_dim = 4
nb_init = 50

In [ ]:
# create conditional vector to generate sample

df_test = pd.read_feather('../data/test_ToU.feather')
df_test['date'] = pd.to_datetime(df_test['date'])
dates = df_test['date']

GMT = []
for d in range(len(dates)):
    for h in range(24):
        GMT = GMT + [dates[d].replace(hour=h), dates[d].replace(hour=h,minute=30)]

d0 = df_test.index[0]
cond = tf.reshape(df_test.loc[d0, cols_cond], shape=[1, cond_dim])
for d in df_test.index[1:len(df_test.index)]:
    cond = tf.concat([cond, tf.reshape(df_test.loc[d, cols_cond], shape=[1, cond_dim])], 0)

In [ ]:
nb_sim = 200

for clustering_name in ['nmf']:
    df_train = pd.read_feather('../data/train_' + clustering_name + '.feather')
    df_train['date'] = pd.to_datetime(df_train['date'])     
    
    if (clustering_name!='Std') & (clustering_name!='ToU'):
        nb_cluster = 4
    else: 
        nb_cluster=1
    for c in range(nb_cluster): 
        if (clustering_name!='Std') & (clustering_name!='ToU'):
            df_train_c = df_train.loc[df_train['cluster'] == c]
        else:
            df_train_c = df_train
            
        conso_max = max(df_train_c.loc[:,cols_data].max())
        conso_min = min(df_train_c.loc[:,cols_data].min())
        
        for it in range(nb_init):
            random.seed(0)
            if (clustering_name=='Std') | (clustering_name=='ToU'):
                model = tf.keras.models.load_model('../data/decoders/' + clustering_name + '_' + str(it) + '.h5', compile=False)
            else:
                model = tf.keras.models.load_model('../data/decoders/' + clustering_name +'cluster_' + str(c) + '_' + str(it) + '.h5', compile=False)
            eps = tf.random.normal(shape=[len(dates),latent_dim], dtype=tf.dtypes.float64)
            conso = simulate_cvae(model, GMT, eps, cond, conso_min, conso_max)
            for _ in range (nb_sim-1):
                eps = tf.random.normal(shape=[len(dates),latent_dim], dtype=tf.dtypes.float64)
                conso = pd.concat([conso,simulate_cvae(model, GMT, eps, cond, conso_min, conso_max)])
            if (clustering_name=='Std') | (clustering_name=='ToU'):
                conso.reset_index(drop=True).to_feather('../data/estimations/cvae_' + clustering_name + '_' + str(it) + '.feather')
            else:
                conso.reset_index(drop=True).to_feather('../data/estimations/cvae_' + clustering_name + '_cluster_' + str(c) + '_' + str(it) + '.feather')